In [1]:
import pandas as pd
import json
import os
from pathlib import Path
import numpy as np

# --- Configuration ---
# Set the path to your data directory, relative to the notebooks folder.
DATA_DIR = Path('D:/NetShieldAI/Data/Processed_CVEs')

# --- Data Loading ---
print(f"Searching for CVE files in: {DATA_DIR}")

all_cve_records = []
# Use rglob to find all files named '*.json' in the directory and its subdirectories
json_files = list(DATA_DIR.rglob('*.json'))

if not json_files:
    print("Error: No JSON files found. Check your DATA_DIR path and folder structure.")
else:
    print(f"Found {len(json_files)} files to process...")

for file_path in json_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            # Assuming the file contains a list of CVE objects
            if isinstance(data, list):
                all_cve_records.extend(data)
            # If the file contains a single CVE object
            elif isinstance(data, dict):
                all_cve_records.append(data)
    except json.JSONDecodeError:
        print(f"Warning: Could not decode JSON from {file_path}. Skipping file.")
    except Exception as e:
        print(f"An error occurred with {file_path}: {e}")


# Convert the list of dictionaries into a pandas DataFrame
if all_cve_records:
    df = pd.DataFrame(all_cve_records)
    print(f"\nSuccessfully loaded {len(df)} total CVE records.")
    print("DataFrame columns:", df.columns.tolist())
    print("\nSample of loaded data:")
    display(df.head())
else:
    print("\nNo data was loaded. Halting execution.")

Searching for CVE files in: D:\NetShieldAI\Data\Processed_CVEs
Found 6 files to process...

Successfully loaded 151849 total CVE records.
DataFrame columns: ['cve_id', 'description', 'keywords', 'platforms', 'affected_products', 'published_date', 'last_modified_date', 'cwe_id', 'base_score', 'severity', 'attack_vector', 'attack_complexity', 'privileges_required', 'user_interaction', 'scope', 'confidentiality_impact', 'integrity_impact', 'availability_impact', 'exploitability_score', 'impact_score']

Sample of loaded data:


,cve_id,description,keywords,platforms,affected_products,published_date,last_modified_date,cwe_id,base_score,severity,attack_vector,attack_complexity,privileges_required,user_interaction,scope,confidentiality_impact,integrity_impact,availability_impact,exploitability_score,impact_score
0,CVE-2020-0001,In getProcessRecordLocked of ActivityManagerSe...,[getprocessrecordlocked activitymanagerservice...,[google:android],"[cpe:2.3:o:google:android:8.0:*:*:*:*:*:*:*, c...",2020-01-08T19:15Z,2024-11-21T04:52Z,NVD-CWE-noinfo,7.8,HIGH,LOCAL,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,N/A,N/A
1,CVE-2020-0002,"In ih264d_init_decoder of ih264d_api.c, there ...","[ih264d_init_decoder ih264d_api, remote code, ...",[google:android],"[cpe:2.3:o:google:android:8.0:*:*:*:*:*:*:*, c...",2020-01-08T19:15Z,2024-11-21T04:52Z,CWE-787,8.8,HIGH,NETWORK,LOW,NONE,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,N/A,N/A
2,CVE-2020-0003,"In onCreate of InstallStart.java, there is a p...","[oncreate installstart, installstart java, pri...",[google:android],[cpe:2.3:o:google:android:8.0:*:*:*:*:*:*:*],2020-01-08T19:15Z,2024-11-21T04:52Z,CWE-367,6.7,MEDIUM,LOCAL,HIGH,LOW,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,N/A,N/A
3,CVE-2020-0004,In generateCrop of WallpaperManagerService.jav...,"[wallpapermanagerservice java, maximum texture...",[google:android],"[cpe:2.3:o:google:android:8.0:*:*:*:*:*:*:*, c...",2020-01-08T19:15Z,2024-11-21T04:52Z,CWE-755,5.5,MEDIUM,LOCAL,LOW,LOW,NONE,UNCHANGED,NONE,NONE,HIGH,N/A,N/A
4,CVE-2020-0005,In btm_read_remote_ext_features_complete of bt...,[btm_read_remote_ext_features_complete btm_acl...,[google:android],"[cpe:2.3:o:google:android:8.0:*:*:*:*:*:*:*, c...",2020-02-13T15:15Z,2024-11-21T04:52Z,CWE-787,6.7,MEDIUM,LOCAL,LOW,HIGH,NONE,UNCHANGED,HIGH,HIGH,HIGH,N/A,N/A


In [2]:
# The 'cwe_id' is essential for our grouping. Drop records where it's missing or marked as 'NVD-CWE-noinfo'.
df.dropna(subset=['cwe_id'], inplace=True)
df = df[df['cwe_id'] != 'NVD-CWE-noinfo']

# Convert numeric columns to the correct data type.
# 'coerce' will turn any non-numeric values into NaN (Not a Number).
df['base_score'] = pd.to_numeric(df['base_score'], errors='coerce')

# For simplicity, we'll focus on the primary impact metrics. Let's create numeric mappings for them.
# HIGH=3, MEDIUM=2, LOW=1, NONE=0
impact_mapping = {'HIGH': 3, 'MEDIUM': 2, 'LOW': 1, 'NONE': 0}
for col in ['confidentiality_impact', 'integrity_impact', 'availability_impact']:
    df[col + '_numeric'] = df[col].map(impact_mapping)

# Drop rows where the base_score is still missing after conversion
df.dropna(subset=['base_score'], inplace=True)

print(f"\nDataFrame shape after cleaning: {df.shape}")
print("Cleaned data types:")
print(df[['cwe_id', 'base_score', 'confidentiality_impact_numeric']].dtypes)


DataFrame shape after cleaning: (99128, 23)
Cleaned data types:
cwe_id                             object
base_score                        float64
confidentiality_impact_numeric    float64
dtype: object


In [3]:
# --- Aggregation ---

# Define the aggregation functions for each feature
# For categorical features, we find the most frequent value (the mode)
# For numerical features, we calculate the mean, max, and standard deviation
agg_functions = {
    'base_score': ['mean', 'max', 'std'],
    'confidentiality_impact_numeric': ['mean'],
    'integrity_impact_numeric': ['mean'],
    'availability_impact_numeric': ['mean'],
    'attack_vector': [lambda x: x.mode()[0] if not x.mode().empty else 'N/A'],
    'privileges_required': [lambda x: x.mode()[0] if not x.mode().empty else 'N/A'],
    'user_interaction': [lambda x: x.mode()[0] if not x.mode().empty else 'N/A'],
    'cve_id': ['count'], # Count how many CVEs fall into this CWE
    'description': [' '.join]
}

print("\nAggregating data by 'cwe_id'...")
cwe_profiles = df.groupby('cwe_id').agg(agg_functions)

# Flatten the multi-level column names
cwe_profiles.columns = ['_'.join(col).strip() for col in cwe_profiles.columns.values]

# Rename the count column for clarity
cwe_profiles.rename(columns={'cve_id_count': 'cve_count'}, inplace=True)

# Fill NaN in 'std' (standard deviation) with 0. This happens for CWEs with only one entry.
cwe_profiles['base_score_std'] = cwe_profiles['base_score_std'].fillna(0)

print("Aggregation complete.")
print(f"Created profiles for {len(cwe_profiles)} unique CWEs.")
print("\nSample of the aggregated CWE profiles:")
display(cwe_profiles.sort_values(by='base_score_mean', ascending=False).head())


Aggregating data by 'cwe_id'...
Aggregation complete.
Created profiles for 307 unique CWEs.

Sample of the aggregated CWE profiles:


,base_score_mean,base_score_max,base_score_std,confidentiality_impact_numeric_mean,integrity_impact_numeric_mean,availability_impact_numeric_mean,attack_vector_<lambda>,privileges_required_<lambda>,user_interaction_<lambda>,cve_count,description_join
cwe_id,,,,,,,,,,,
CWE-566,10.0,10.0,0.0,3.0,3.0,3.0,NETWORK,NONE,NONE,1,Authorization Bypass Through User-Controlled S...
CWE-420,10.0,10.0,0.0,3.0,3.0,3.0,NETWORK,NONE,NONE,1,Snap One OvrC cloud servers contain a route an...
CWE-656,9.8,9.8,0.0,3.0,3.0,3.0,NETWORK,NONE,NONE,1,The devices are vulnerable to an authenticatio...
CWE-305,9.8,9.8,0.0,3.0,3.0,3.0,NETWORK,NONE,NONE,1,MileSight DeviceHub - \n\n\n\nCWE-305 Missing ...
CWE-1285,9.8,9.8,0.0,3.0,3.0,3.0,NETWORK,NONE,NONE,1,IBM Tivoli Monitoring 6.3.0.7 through 6.3.0.7 ...


In [4]:
# --- Feature Engineering: Actual Risk Score ---

# Define the weights for our custom score
AV_WEIGHTS = {'NETWORK': 1.5, 'ADJACENT_NETWORK': 1.2, 'LOCAL': 1.0, 'PHYSICAL': 0.8}
PR_WEIGHTS = {'NONE': 1.2, 'LOW': 1.0, 'HIGH': 0.8}

# Map the weights to the aggregated columns
cwe_profiles['av_weight'] = cwe_profiles['attack_vector_<lambda>'].map(AV_WEIGHTS).fillna(1.0)
cwe_profiles['pr_weight'] = cwe_profiles['privileges_required_<lambda>'].map(PR_WEIGHTS).fillna(1.0)

# Calculate the score
cwe_profiles['actual_risk_score'] = cwe_profiles['base_score_mean'] * cwe_profiles['av_weight'] * cwe_profiles['pr_weight']

# Let's see the most dangerous vulnerability types according to our new score
print("\nTop 10 Vulnerability Types by 'Actual Risk Score':")
display(cwe_profiles.sort_values(by='actual_risk_score', ascending=False).head(10))

# --- Save the Result ---
# Save the final processed data to a file for the next phase
output_path = Path('../Data/cwe_profiles.csv')
cwe_profiles.to_csv(output_path)

print(f"\nPhase 1 complete! The processed data has been saved to:\n{output_path.resolve()}")


Top 10 Vulnerability Types by 'Actual Risk Score':


,base_score_mean,base_score_max,base_score_std,confidentiality_impact_numeric_mean,integrity_impact_numeric_mean,availability_impact_numeric_mean,attack_vector_<lambda>,privileges_required_<lambda>,user_interaction_<lambda>,cve_count,description_join,av_weight,pr_weight,actual_risk_score
cwe_id,,,,,,,,,,,,,,
CWE-420,10.000000,10.0,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,Snap One OvrC cloud servers contain a route an...,1.5,1.2,18.0000
CWE-566,10.000000,10.0,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,Authorization Bypass Through User-Controlled S...,1.5,1.2,18.0000
CWE-1285,9.800000,9.8,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,IBM Tivoli Monitoring 6.3.0.7 through 6.3.0.7 ...,1.5,1.2,17.6400
CWE-656,9.800000,9.8,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,The devices are vulnerable to an authenticatio...,1.5,1.2,17.6400
CWE-648,9.800000,9.8,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,The D-Link DSL6740C modem has an Incorrect Use...,1.5,1.2,17.6400
CWE-305,9.800000,9.8,0.000000,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,1,MileSight DeviceHub - \n\n\n\nCWE-305 Missing ...,1.5,1.2,17.6400
CWE-114,9.250000,10.0,0.818535,3.000000,3.000000,3.000000,NETWORK,NONE,NONE,4,IBM AIX 7.2 and 7.3 nimesis NIM master service...,1.5,1.2,16.6500
CWE-917,9.135556,10.0,1.062995,2.911111,2.822222,2.833333,NETWORK,NONE,NONE,90,Sonatype Nexus Repository before 3.21.2 allows...,1.5,1.2,16.4440
CWE-406,9.100000,9.1,0.000000,3.000000,0.000000,3.000000,NETWORK,NONE,NONE,1,"RTI Connext DDS Professional, Connext DDS Secu...",1.5,1.2,16.3800



Phase 1 complete! The processed data has been saved to:
D:\NetShieldAI\Data\cwe_profiles.csv
